- [ ] Write down all the libraries that we need to install
- [ ] Add Try catch for exceptions when moving to .py file

## Important Notes
### Packages to install :
- pip install sounddevice
- pip install numpy
- Install ffmpeg (needed for intsalling local version of whisper)
    - On Ubuntu or Debian
      sudo apt update && sudo apt install ffmpeg
    - On Arch Linux
      sudo pacman -S ffmpeg
    - On MacOs
      brew install ffmpeg
    - On Windows
      winget install ffmpeg
- pip install -U openai-whisper
- pip install anthropic
- pip install dotenv
- pip install python-dotenv
- pip install openai
- pip install pyaudio
- pip install keyboard
- Installing pytorch
  - Install Cuda Toolkit (https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html) -> Only needed for GPU acceleration
  - Install Cudnn (https://developer.nvidia.com/cudnn) -> Only needed for GPU acceleration
  - pInstall pytorch with or without cuda (https://pytorch.org/get-started/locally/)
- pip install PyMuPDF -> To read the pdf files like CV's
### Instructions :
- There needs to be a .env in this notebook's working directory, which contains the api keys.

In [1]:
# Imports
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import whisper
from joblib import load, dump
from AnthropicWrapper import ClaudeChatCV
from dotenv import load_dotenv, find_dotenv
import os
from openai import OpenAI
import pyaudio
import threading
import time
import keyboard
import re

In [2]:
# Get the path to the immediate parent folder of the current working directory
parent_folder_path = os.path.dirname(os.getcwd())

# Construct the path to the .env file in the parent folder
dotenv_path = os.path.join(parent_folder_path, ".env")

# Load the .env file
load_dotenv(dotenv_path)

True

In [3]:
# --- Settings ---
FS = 44100               # Sampling frequency
THRESHOLD = 50          # Volume threshold for silence (adjust this)
SILENCE_DURATION = 1.5   # Seconds of silence before stopping (adjust this)
CHUNK_SIZE = 1024        # Process audio in chunks for efficiency
SPEAKING_SPEED = 1.1     # Speed of speaking

In [4]:
# --- Globals ---
pause_loop = False
audio_data = np.array([], dtype=np.int16)

In [5]:
job_role = "Machine Learning Engineer"
candidate_skill = "Entry-Level"

role_description = """
Do you want to tackle the biggest questions in finance with near infinite compute power at your fingertips?

G-Research is a leading quantitative research and technology firm, with offices in London and Dallas. We are proud to employ some of the best people in their field and to nurture their talent in a dynamic, flexible and highly stimulating culture where world-beating ideas are cultivated and rewarded.

This is a role based in our new Soho Place office - opened in 2023 - in the heart of Central London and home to our Research Lab.

The role

We are looking for exceptional machine learning engineers to work alongside our quantitative researchers on cutting-edge machine learning problems.

As a member of the Core Technical Machine Learning team, you will be engaged in a mixture of individual and collaborative work to tackle some of the toughest research questions.

In this role, you will use a combination of off-the-shelf tools and custom solutions written from scratch to drive the latest advances in quantitative research.

Past projects have included:

Implementing ideas from a recently published research paper
Writing custom libraries for efficiently training on petabytes of data
Reducing model training times by hand optimising machine learning operations
Profiling custom ML architectures to identify performance bottlenecks
Evaluating the latest hardware and software in the machine learning ecosystem
Who are we looking for?

Candidates will be comfortable working both independently and in small teams on a variety of engineering challenges, with a particular focus on machine learning and scientific computing.

The ideal candidate will have the following skills and experience:

Either a post-graduate degree in machine learning or a related discipline, or commercial experience working on machine learning models at scale. We will also consider exceptional candidates with a proven record of success in online data science competitions, such as Kaggle
Strong object-oriented programming skills and experience working with Python, PyTorch and NumPy are desirable
Experience in one or more advanced optimisation methods, modern ML techniques, HPC, profiling, model inference; you dont need to have all of the above
Excellent ML reasoning and communication skills are crucial: off-the-shelf methods dont always work on our data so you will need to understand how to develop your own models in a collaborative environment working in a team with complementary skills
Finance experience is not necessary for this role and candidates from non-financial backgrounds are encouraged to apply.

Why should you apply?

Highly competitive compensation plus annual discretionary bonus
Lunch provided (via Just Eat for Business) and dedicated barista bar
35 days annual leave
9 percent company pension contributions
Informal dress code and excellent work/life balance
Comprehensive healthcare and life assurance
Cycle-to-work scheme
Monthly company events
"""

In [6]:
system_prompt = f"""You are a skilled interviewer who is conducting an initial phone screening interview for a candidate for a {candidate_skill} {job_role} role to see if the candidate is at minimum somewhat qualified for the role and worth the time to be fully interviewed. The role and company description is copypasted from the job posting as follows: {role_description}. Parse through it to extract any information you feel is relevant.
Your job is to begin a friendly discussion with the candidate, and ask questions relevant to the {job_role} role, which may or may not be based on the interviewee's CV, which you have access to. Be sure to stick to this topic even if the candidate tries to steer the conversation elsewhere. If the candidate has other experience on his CV, you can ask about it, but keep it within the context of the {job_role} role.
After the candidate responds to each of your questions, you should not summarise or provide feedback on their responses. THIS POINT IS KEY! You should not summarise or provide feedback on their responses. You must keep your responses short and concise without reiterating what is good about the candidate's response or experience when they reply.
You can ask follow-up questions if you wish.
Once you have asked sufficient questions such that you deem the candidate is or isn't fitting for the role, end the interview by using the specific phrase thank you for your time and informing them that they will receive word soon on the outcome of the screening interview. If the candidate does not seem fititng for the role, or if something feels off such as the candidate being unconfident or very very vague feel free to end the interview early. There is no need to inform them of your opinion of their performance, as this will be evaluated later.
The candidate will begin the interview by greeting you. You are to greet them back, and begin the interview.
For this specific run, keep the interview to a maximum of 3 questions."""

In [7]:
# Initialising Claude and ConversationChain
chat_model_name = "claude-3-5-sonnet-20240620"

# Give the path to a CV in your disk
pdf_path = r"D:\Kent\University Of Kent UK\Jobs\CV-DebapratimKundu.pdf"
chat_model = ClaudeChatCV(chat_model_name, system_prompt, pdf_path)

In [8]:
# Initialising whisper
stt_model = whisper.load_model("medium", device="cuda")

In [9]:
# Initialising text2speech
tts_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def stream_tts(input_string):
    def _stream_tts():
        p = pyaudio.PyAudio()
        stream = p.open(format=8,
                        channels=1,
                        rate=round(24_005 * SPEAKING_SPEED),
                        output=True)
        with tts_client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice="nova",
            input=input_string,
            response_format="pcm"
        ) as response:
            for chunk in response.iter_bytes(1024):
                stream.write(chunk)
                
        #print("FINISHED!!!!!!!!!!!!!!!!!!!!")
        thread_done.set()

    thread_done = threading.Event()

    thread = threading.Thread(target=_stream_tts)
    thread.start()
    thread_done.wait()

In [10]:
# --- Functions ---
def is_silent(data):
    rms = np.sqrt(np.mean(data**2))
    print("RMS: ", rms)
    return rms < THRESHOLD

In [11]:
# filename = input("Enter the filename of the conversation: ")
filename = "interview_audio"

In [12]:
# Initialising speech2text without silence detection
def record_speech():
    global audio_data
    print ("Printing audio data...")
    print (audio_data)
    print("Recording... Speak now!")
    new_audio = np.array([], dtype=np.int16)  # Initialize empty array

    with sd.InputStream(samplerate=FS, channels=1, dtype='int16') as stream:
        while True:
            chunk, overflowed = stream.read(CHUNK_SIZE)
            if overflowed:
                print("Warning: Input overflowed!")
            new_audio = np.append(new_audio, chunk)

            if pause_loop:
                break
    audio_data = np.append(audio_data, new_audio)
    
    wav.write("temp_audio.wav", FS, new_audio)
    wav.write(f"{filename}.wav", FS, audio_data)
    return "temp_audio.wav"


In [13]:
def keyboard_listener():
    global pause_loop
    
    def on_key_press(event):
        global pause_loop
        #print("Key pressed: {}" .format(event.name))
        if event.name == "+":
            pause_loop = False
            print("Set to continue on next loop")
        elif event.name == "-":
            pause_loop = True
            print("Set to pause on next loop")
    
    keyboard.on_press(on_key_press)
    keyboard.wait('esc')

listener_thread = threading.Thread(target=keyboard_listener)
listener_thread.start()

In [14]:
while True:
    if not pause_loop:
        # --- Record Speech ---
        time.sleep(0.1)
        print("Recording speech...")
        wav_file = record_speech()

        # --- Speech to Text ---
        print("Converting speech to text...")
        text = stt_model.transcribe(wav_file, language="en")
        print("You said: ", text.get("text"))

        # --- Chatbot ---
        print("Chatting...")
        response = chat_model.chat_with_history_doc(text.get("text"))

        print("Chatbot: ", response)

        # --- Text to Speech ---
        print("Converting text to speech...")
        stream_tts(response)
        # Check if the response indicates the end of the interview
        if re.search(r"thank you for your time|interview (is )?over|that concludes our interview", response.lower()):
            print("Interview concluded. Exiting in 5 seconds...")
            time.sleep(5)
            break

    else:
        time.sleep(0.1)

Recording speech...
Printing audio data...
[]
Recording... Speak now!
Converting speech to text...
You said:   Hello.
Chatting...
Chatbot:  Hello! Thank you for taking the time to speak with me today about the Entry-Level Machine Learning Engineer position at G-Research. To start, could you tell me about your experience with machine learning, particularly any projects or coursework you've completed in this area?
Converting text to speech...
Recording speech...
Printing audio data...
[   0    0   -1 ... -198 -256  -40]
Recording... Speak now!
Converting speech to text...
You said:   I'm sure I have a total of six months of experience with machine learning after I entered the University of Kent. Right now I'm mainly working on Numpy and Pandas. Numpy is to analyze image information and Pandas to extract data from CSVs.
Chatting...
Chatbot:  Thank you for sharing that information. Could you elaborate on a specific machine learning project you've worked on, perhaps one that involved Numpy 

Set to pause on next loop
Set to continue on next loop
Set to pause on next loop
Set to continue on next loop
Set to pause on next loop
Set to continue on next loop
Set to pause on next loop


In [15]:
conversation = chat_model.get_conversation()
dump(conversation, "conversation.joblib")
# Dump the conversation with joblib


print(conversation)
import re
from fpdf import FPDF
# Function to extract role and content from each turn
def parse_turn(turn):
    role_match = re.search(r"role: (.*)", turn)
    content_match = re.search(r"content: (.*)", turn, re.DOTALL)

    role = role_match.group(1) if role_match else ""
    content = content_match.group(1).strip() if content_match else turn.strip()

    return role, content

# Create a list of dictionaries, each representing a turn in the conversation
conversation_list = []
for turn in conversation.split("--------------------\n"):
    if turn.strip():  # Ignore empty turns
        role, content = parse_turn(turn)
        #print(content)
        if "Understood! I'll keep this CV in the back of my mind and use it should it be relevant to the discussion." not in content and "The following is a CV that I am providing you with." not in content:
            conversation_list.append({"role": role, "content": content})

# Create a PDF object
pdf = FPDF()
pdf.add_page()

# Set margins (in millimeters)
pdf.set_margins(left=10, top=20, right=10)  # Right margin set to 10mm 

# Choose a nicer font
pdf.set_font("Helvetica", size=12)

# Calculate usable width for text wrapping (accounting for margins)
usable_width = pdf.w - pdf.l_margin - pdf.r_margin -0 # 10px right margin 

# Add conversation to the PDF
for turn in conversation_list:
    # Skip turns with empty roles or content
    if turn['role'] and turn['content']:
        # Subheading style for "User" and "Assistant"
        pdf.set_font("Helvetica", style="B", size=14)
        pdf.cell(0, 10, txt=f"{turn['role'].capitalize()}:", ln=True)

        # Content with regular font and correct indentation
        pdf.set_font("Helvetica", size=12)
        pdf.x = pdf.l_margin  # Reset x-coordinate to the left margin
        pdf.multi_cell(usable_width, 6, txt=turn['content'])
        pdf.ln(3)

# Save the PDF
pdf.output("conversation.pdf")

role: user
content: The following is a CV that I am providing you with. You are to keep this document in the back of your mind and consider it or use it, should it be relevant to the discussion.

 Document below: 



## Debapratim Kundu 
debo121@live.com || E8 E Woolf College, Giles Lane, Canterbury, CT2 7BQ || +44 7393062352 
LinkedIn: https://www.linkedin.com/in/debapratim-kundu-95a089183 ||  
Git: https://github.com/DKundu121?tab=repositories 
 
I am currently pursuing a master’s degree in Artificial Intelligence from the University of Kent. I have 
over 5 years of experience in designing, developing and maintaining web applications. I have 
successfully delivered solutions for domains such as e-commerce, finance and retail. I am passionate 
about finding insights from data and solving complex problems. I am looking for a challenging and 
rewarding opportunity to apply the new skills I have picked up and to learn new ones. 
 
WORK EXPERIENCE: 
 
TCS 
Developer/Tester                

''